<a href="https://colab.research.google.com/github/Aswini-J/EXP-6---Matrix-multiplication-using-cuBLAS-in-CUDA-C-/blob/main/Matrix_multiplication_using_cuBLAS_in_CUDA_C_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4tr5xwpg
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4tr5xwpg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-cvrs62tu/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp4uyppqk6".


In [1]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/u

In [3]:
# Write CUDA C code to a .cu file
code = """
#include <stdlib.h>
#include <stdio.h>
#include <cublas_v2.h>
#include <cuda_runtime.h>

#define HA 2
#define WA 9
#define WB 2
#define HB WA
#define WC WB
#define HC HA
#define index(i,j,ld) (((j)*(ld))+(i))

void printMat(float *P, int uWP, int uHP) {
    for (int i = 0; i < uHP; i++) {
        for (int j = 0; j < uWP; j++) {
            printf("%f ", P[index(i, j, uHP)]);
        }
        printf("\\n");
    }
}

__host__ float* initializeHostMemory(int height, int width, bool random, float value) {
    float *hostMatrix = (float*)malloc(height * width * sizeof(float));
    for (int i = 0; i < height * width; i++) {
        hostMatrix[i] = random ? rand() / (float)RAND_MAX : value;
    }
    return hostMatrix;
}

__host__ float* initializeDeviceMemoryFromHostMemory(int height, int width, float *hostMatrix) {
    float *deviceMatrix;
    cudaMalloc((void**)&deviceMatrix, height * width * sizeof(float));
    cudaMemcpy(deviceMatrix, hostMatrix, height * width * sizeof(float), cudaMemcpyHostToDevice);
    return deviceMatrix;
}

__host__ float* retrieveDeviceMemory(int height, int width, float *deviceMatrix, float *hostMemory) {
    cudaMemcpy(hostMemory, deviceMatrix, height * width * sizeof(float), cudaMemcpyDeviceToHost);
    return hostMemory;
}

int main() {
    float *A = initializeHostMemory(HA, WA, true, 0.0f);
    float *B = initializeHostMemory(HB, WB, true, 0.0f);
    float *C = (float*)malloc(HC * WC * sizeof(float));

    float *AA = initializeDeviceMemoryFromHostMemory(HA, WA, A);
    float *BB = initializeDeviceMemoryFromHostMemory(HB, WB, B);
    float *CC = initializeDeviceMemoryFromHostMemory(HC, WC, C);

    float alpha = 1.0f;
    float beta = 0.0f;




     // Write Your code here




    printf("Matrix A:\\n");
    printMat(A, WA, HA);
    printf("\\nMatrix B:\\n");
    printMat(B, WB, HB);
    printf("\\nMatrix C:\\n");
    printMat(C, WC, HC);

    cublasDestroy(handle);
    cudaFree(AA);
    cudaFree(BB);
    cudaFree(CC);
    free(A);
    free(B);
    free(C);

    return 0;
}
"""

with open("matrix_multiplication.cu", "w") as file:
    file.write(code)


In [4]:
!nvcc matrix_multiplication.cu -lcublas -o matrix_multiplication

In [5]:
!./matrix_multiplication


Matrix A:
0.840188 0.783099 0.911647 0.335223 0.277775 0.477397 0.364784 0.952230 0.635712 
0.394383 0.798440 0.197551 0.768230 0.553970 0.628871 0.513401 0.916195 0.717297 

Matrix B:
0.141603 0.108809 
0.606969 0.998924 
0.016301 0.218257 
0.242887 0.512932 
0.137232 0.839112 
0.804177 0.612640 
0.156679 0.296032 
0.400944 0.637552 
0.129790 0.524287 

Matrix C:
1.493319 2.564243 
2.352294 2.700922 
